In [1]:
# Importar librerías necesarias
import pandas as pd

In [2]:
# Definir la ruta al archivo CSV si no está definida
file_path_precio_m2 = '/Users/patriciajaquez/Documents/Documents - Patricia’s MacBook Pro/GitHub/Analisis-de-inversion-inmobiliario/data/malaga_precio_m2_raw.csv'

# Cargar el archivo malaga_precio_m2.csv usando la variable ya definida
try:
    precio_m2_df = pd.read_csv(file_path_precio_m2, encoding='utf-8')
except UnicodeDecodeError:
    precio_m2_df = pd.read_csv(file_path_precio_m2, encoding='latin1')

# Buscar el nombre de la columna con "Mes" y "A" para renombrar correctamente
historical_max_date_col = [col for col in precio_m2_df.columns if "Mes" in col and "A" in col]
rename_dict = {
    'Localizacion': 'neighbourhood',
    'Precio m2 mayo 2025': 'price_per_m2',
    'Variacion mensual': 'monthly_variation',
    'Variacion trimestral': 'quarterly_variation',
    'Variacion anual': 'annual_variation',
    'Maximo Historico': 'historical_max',
    'Variacion maximo': 'historical_max_variation'
}
if historical_max_date_col:
    rename_dict[historical_max_date_col[0]] = 'historical_max_date'

precio_m2_df.rename(columns=rename_dict, inplace=True)

# Definir las columnas a limpiar
columns_to_clean = ['monthly_variation', 'quarterly_variation', 'annual_variation', 'historical_max_variation']

# Limpiar valores numéricos solo si la columna existe, manteniendo los valores negativos
for col in columns_to_clean:
    if col in precio_m2_df.columns:
        precio_m2_df[col] = (
            precio_m2_df[col]
            .astype(str)
            .str.replace(',', '.', regex=False)  # Reemplazar comas por puntos
            .str.replace('%', '', regex=False)  # Eliminar el símbolo de porcentaje
            .str.replace('+', '', regex=False)  # Eliminar el símbolo de positivo
            .str.replace('−', '-', regex=False)  # Reemplazar el signo negativo Unicode por el estándar
            .str.replace(' ', '', regex=False)  # Eliminar espacios innecesarios
            .str.strip()  # Eliminar espacios adicionales
        )
        # Convertir a numérico, manteniendo los valores negativos
        precio_m2_df[col] = pd.to_numeric(precio_m2_df[col], errors='coerce')

# Limpiar nombres de zonas para que coincidan con malaga_clean.csv
if 'neighbourhood' in precio_m2_df.columns:
    precio_m2_df['neighbourhood'] = precio_m2_df['neighbourhood'].astype(str).str.strip()

# Mostrar el DataFrame limpio
print("DataFrame limpio:")
display(precio_m2_df.head())

DataFrame limpio:


,neighbourhood,price_per_m2,monthly_variation,quarterly_variation,annual_variation,historical_max,historical_max_date,historical_max_variation
0,Malaga,3427,1.4,3.8,17.4,3.427,mayo 2025,0.0
1,Bailen - Miraflores,2849,0.4,6.5,33.2,2.849,mayo 2025,0.0
2,Campanillas,2165,4.8,5.6,4.8,2.268,diciembre 2024,-4.5
3,Carretera de Cadiz,3491,0.6,1.1,12.3,3.491,mayo 2025,0.0
4,Centro,4017,-0.4,2.2,8.3,4.035,abrirl 2025,-0.4


In [3]:
precio_m2_df.info()  # Mostrar información del DataFrame limpio

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   neighbourhood             10 non-null     object 
 1   price_per_m2              10 non-null     int64  
 2   monthly_variation         10 non-null     float64
 3   quarterly_variation       10 non-null     float64
 4   annual_variation          10 non-null     float64
 5   historical_max            10 non-null     float64
 6   historical_max_date       10 non-null     object 
 7   historical_max_variation  10 non-null     float64
dtypes: float64(5), int64(1), object(2)
memory usage: 772.0+ bytes


In [4]:
# Corregir tipos de datos para columnas numéricas

# Convertir price_per_m2 a float
precio_m2_df['price_per_m2'] = pd.to_numeric(precio_m2_df['price_per_m2'], errors='coerce')

# Eliminar puntos de 'historical_max'
precio_m2_df['historical_max'] = (
    precio_m2_df['historical_max']
    .astype(str)
    .str.replace('.', '', regex=False)  # eliminar puntos
)

# Redondear columnas floats a 2 decimales
float_columns = ['price_per_m2', 'monthly_variation', 'quarterly_variation', 'annual_variation', 'historical_max', 'historical_max_variation']
for col in float_columns:
    if col in precio_m2_df.columns:
        # Asegurarse de que los valores sean numéricos antes de redondear
        precio_m2_df[col] = (
            precio_m2_df[col]
            .astype(str)  # Convertir a string para limpieza
            .str.replace(',', '.', regex=False)  # Reemplazar comas por puntos
            .str.replace('%', '', regex=False)  # Eliminar el símbolo de porcentaje
            .str.replace('+', '', regex=False)  # Eliminar el símbolo de positivo
            .str.replace('−', '-', regex=False)  # Asegurar que los signos negativos sean correctos
            .str.strip()  # Eliminar espacios innecesarios
        )
        # Convertir a numérico y redondear a 2 decimales
        precio_m2_df[col] = pd.to_numeric(precio_m2_df[col], errors='coerce').round(2)

# Verificar los cambios realizados
print("DataFrame después de las correcciones:")
precio_m2_df.info() 

DataFrame después de las correcciones:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   neighbourhood             10 non-null     object 
 1   price_per_m2              10 non-null     int64  
 2   monthly_variation         10 non-null     float64
 3   quarterly_variation       10 non-null     float64
 4   annual_variation          10 non-null     float64
 5   historical_max            10 non-null     int64  
 6   historical_max_date       10 non-null     object 
 7   historical_max_variation  10 non-null     float64
dtypes: float64(4), int64(2), object(2)
memory usage: 772.0+ bytes


In [5]:
# Imprimir las primeras filas del DataFrame corregido
display(precio_m2_df.head())

,neighbourhood,price_per_m2,monthly_variation,quarterly_variation,annual_variation,historical_max,historical_max_date,historical_max_variation
0,Malaga,3427,1.4,3.8,17.4,3427,mayo 2025,0.0
1,Bailen - Miraflores,2849,0.4,6.5,33.2,2849,mayo 2025,0.0
2,Campanillas,2165,4.8,5.6,4.8,2268,diciembre 2024,-4.5
3,Carretera de Cadiz,3491,0.6,1.1,12.3,3491,mayo 2025,0.0
4,Centro,4017,-0.4,2.2,8.3,4035,abrirl 2025,-0.4


In [6]:
# Guardar el archivo limpio
output_path_precio_m2 = '/Users/patriciajaquez/Documents/Documents - Patricia’s MacBook Pro/GitHub/Analisis-de-inversion-inmobiliario/data/malaga_precio_m2_clean.csv'
precio_m2_df.to_csv(output_path_precio_m2, index=False)

print(f"Archivo limpio guardado en: {output_path_precio_m2}")

Archivo limpio guardado en: /Users/patriciajaquez/Documents/Documents - Patricia’s MacBook Pro/GitHub/Analisis-de-inversion-inmobiliario/data/malaga_precio_m2_clean.csv
